In [ ]:
import validate_messages
import json
from set_environment import config
import pprint
from azure.servicebus import ServiceBusClient
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from var_funcs import CREDENTIAL
from pins_data_model import load_schemas
import pytest

In [ ]:
_ENTITY = "service-user"
_CREDENTIAL = CREDENTIAL
_NAMESPACE = config['dev']["servicebus_namespace_odt"]
_MAX_MESSAGE_COUNT = config["global"]["max_message_count"]
_SCHEMAS = load_schemas.load_all_schemas()["schemas"]
_SCHEMA = _SCHEMAS[f"{_ENTITY}.schema.json"]
_TOPIC = config["global"]["entities"][f"{_ENTITY}"]["topic"]
_SUBSCRIPTION = config["global"]["entities"][f"{_ENTITY}"]["subscription"]

In [ ]:
def read_messages(namespace: str,
    credential: DefaultAzureCredential,
    topic: str,
    subscription: str,
    max_message_count: int,
) -> list:

    messages = []

    servicebus_client = ServiceBusClient(
    fully_qualified_namespace=namespace, credential=credential
)
    
    with servicebus_client:
        subscription_receiver = servicebus_client.get_subscription_receiver(
            topic_name=topic, 
            subscription_name=subscription,
        )

        print("Reading messages...")

        with subscription_receiver:
            received_msgs = subscription_receiver.peek_messages(
                max_message_count
            )

            for message in received_msgs:
                message_body = json.loads(str(message))
                messages.append(message_body)

    return messages

In [ ]:
data = read_messages(_NAMESPACE, _CREDENTIAL, _TOPIC, _SUBSCRIPTION, _MAX_MESSAGE_COUNT)
pprint.pprint(data)

In [ ]:
validate_messages.validate_data(data, schema=_SCHEMA)

#### Employee

In [ ]:
schema: dict = {
	"$schema": "http://json-schema.org/draft-07/schema#",
	"$id": "employee.schema.json",
	"title": "Employee",
	"type": "object",
	"required": ["id", "firstName", "lastName"],
	"properties": {
		"id": {
			"type": "string"
		},
		"firstName": {
			"type": "string"
		},
		"lastName": {
			"type": "string"
		},
        "my_optional_field": {
            "type": ["string"]
		}
	}
}

In [ ]:
data: list[dict] = [{
        "id": "12345",
        "firstName": "Chris",
        "lastName": "Topping",
        "my_optional_field": None
    }
]

In [ ]:
validate_messages.validate_data(data, schema)